In [0]:
# %pip install numpy pandas matplotlib torch scipy tqdm plotly optuna
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter as TorchParam
from torch import Tensor
from typing import List, Tuple

from scipy.stats import norm
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

import optuna
from torch.jit import ScriptModule, script_method

##Data setup

In [0]:
MeasurementData_Merged = pd.read_csv("MeasurementData_Merged_EFAD.csv")
# MeasurementData_Merged = MeasurementData_Merged.rename(columns={'Rotor Temperature TelemetriesExternTemp.rp_rbe_Cif_10ms_PIExternTemp1.rbe_Cif': 'ExternTemp.rp_rbe_Cif_10ms_PIExternTemp1.rbe_Cif', 'Stator Temperature TelemetriesI_EM_tTMotWinDeLay2Cl03': 'I_EM_tTMotWinDeLay2Cl03'})
# MeasurementData_Merged.to_pickle('MeasurementData_Merged.pkl')
MeasurementData_Merged['pm'] = MeasurementData_Merged[['ExternTemp.rp_rbe_Cif_10ms_PIExternTemp1.rbe_Cif', 'ExternTemp.rp_rbe_Cif_10ms_PIExternTemp10.rbe_Cif', 'ExternTemp.rp_rbe_Cif_10ms_PIExternTemp5.rbe_Cif','ExternTemp.rp_rbe_Cif_10ms_PIExternTemp7.rbe_Cif']].mean(axis=1)
MeasurementData_Merged['stator_winding'] = MeasurementData_Merged[['I_EM_tTMotWinDeLay2Cl03', 'I_EM_tTMotWinDeLay2Cl12','I_EM_tTMotWinDeLay5Cl03','I_EM_tTMotWinDeLay5Cl06','I_EM_tTMotWinDeLay5Cl12',
                                                                   'I_EM_tTMotWinNdeLay2Cl03','I_EM_tTMotWinNdeLay2Cl12','I_EM_tTMotWinNdeLay5Cl03']].mean(axis=1)
MeasurementData_Merged['Us'] = np.sqrt(MeasurementData_Merged['uDaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm']**2 + MeasurementData_Merged['uQaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm']**2)
MeasurementData_Merged['Is'] = np.sqrt(MeasurementData_Merged['iDaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm']**2 + MeasurementData_Merged['iQaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm']**2)

# input_cols = ['nEmFild100ms.Rec_10ms_Fild.pp_rbe_CddAgEm_10ms_Fild.rbe_CddRslvr',
#                 'tqEmFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
#                 'iDaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm',
#                 'iQaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm',
#                 'uDaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
#                 'uQaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
#                 'R_CW_tCooltIvtrOut',
#                 # 'R_EM_tTMotRshaftOilIn',
#                 'tEmSnsrFild10ms.Rec_2ms_Fild.rp_rbe_CddTEm_2ms_Fild.rbe_TMdlEm',
#                 'Us',
#                 'Is']

input_cols = ['nEmFild100ms.Rec_10ms_Fild.pp_rbe_CddAgEm_10ms_Fild.rbe_CddRslvr',
                'tqEmFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
                'iDaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm',
                'iQaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm',
                'uDaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
                'uQaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
                'R_CW_tCooltIvtrOut',
                # 'R_EM_tTMotRshaftOilIn',
                'tEmSnsrFild10ms.Rec_2ms_Fild.rp_rbe_CddTEm_2ms_Fild.rbe_TMdlEm']

In [0]:
# path_to_csv = Path().cwd() / "data" / "input" / "measures_v2.csv"
data = MeasurementData_Merged.copy()
data = data[data['pm'] <= 200]
# data = data[data['stator_winding'] <= 150]
# target_cols = ['pm', 'stator_winding']
# target_cols = ['pm']
# target_cols = ['stator_winding']
target_cols = ['pm','stator_winding','R_EM_tTMotRshaftOilIn']
temperature_cols = target_cols + ['tEmSnsrFild10ms.Rec_2ms_Fild.rp_rbe_CddTEm_2ms_Fild.rbe_TMdlEm','R_CW_tCooltIvtrOut']
test_profiles = [36, 37]
train_profiles = [p for p in data.profile_id.unique() if p not in test_profiles]
profile_sizes = data.groupby("profile_id").agg("size")

# normalize
non_temperature_cols = [c for c in data if c in input_cols and c not in temperature_cols]
data.loc[:, temperature_cols] /= 200  # deg C
data.loc[:, non_temperature_cols] /= data.loc[:, non_temperature_cols].abs().max(axis=0)

# # extra feats (FE)
# if {"i_d", "i_q", "u_d", "u_q"}.issubset(set(data.columns.tolist())):
#     extra_feats = {
#         "i_s": lambda x: np.sqrt((x["i_d"] ** 2 + x["i_q"] ** 2)),
#         "u_s": lambda x: np.sqrt((x["u_d"] ** 2 + x["u_q"] ** 2)),
#     }
# data = data.assign(**extra_feats)
# input_cols = [c for c in data.columns if c not in target_cols]
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# overwrite. We recommend CPU over GPU here, as that runs faster with pytorch on this data set
device = torch.device("cpu")

In [0]:
# Rearrange features
# input_cols = [c for c in data.columns if c not in target_cols + ["profile_id"]]
data = data.loc[:, input_cols + ["profile_id"] + target_cols].dropna()

def generate_tensor(profiles_list):
    """Returns profiles of the data set in a coherent 3D tensor with
    time-major shape (T, B, F) where
    T : Maximum profile length
    B : Batch size = Amount of profiles
    F : Amount of input features.

    Also returns a likewise-shaped sample_weights tensor, which zeros out post-padded zeros for use
    in the cost function (i.e., it acts as masking tensor)"""
    tensor = np.full(
        (profile_sizes[profiles_list].max(), len(profiles_list), data.shape[1] - 1),
        np.nan,
    )
    for i, (pid, df) in enumerate(
        data.loc[data.profile_id.isin(profiles_list), :].groupby("profile_id")
    ):
        assert pid in profiles_list, f"PID is not in {profiles_list}!"
        tensor[: len(df), i, :] = df.drop(columns="profile_id").to_numpy()
    sample_weights = 1 - np.isnan(tensor[:, :, 0])
    tensor = np.nan_to_num(tensor).astype(np.float32)
    tensor = torch.from_numpy(tensor).to(device)
    sample_weights = torch.from_numpy(sample_weights).to(device)
    return tensor, sample_weights


train_tensor, train_sample_weights = generate_tensor(train_profiles)
test_tensor, test_sample_weights = generate_tensor(test_profiles)

## Model declaration

In [0]:
class DiffEqLayer(nn.Module):
    """This class is a container for the computation logic in each step.
    This layer could be used for any 'cell', also RNNs, LSTMs or GRUs."""

    def __init__(self, cell, *cell_args):
        super().__init__()
        self.cell = cell(*cell_args)

    def forward(self, input: Tensor, state: Tensor) -> Tuple[Tensor, Tensor]:
        inputs = input.unbind(0)
        outputs = torch.jit.annotate(List[Tensor], [])
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs += [out]
        return torch.stack(outputs), state


class TNNCell(nn.Module):
    """The main TNN logic. Here, the sub-NNs are initialized as well as the constant learnable
    thermal capacitances. The forward function houses the LPTN ODE discretized with the explicit Euler method
    """

    def __init__(self):
        super().__init__()
        self.sample_time = 0.5  # in s
        self.output_size = len(target_cols)
        self.caps = TorchParam(torch.Tensor(self.output_size).to(device))
        nn.init.normal_(
            self.caps, mean=-9.2, std=0.5
        )  # hand-picked init mean, might be application-dependent
        n_temps = len(temperature_cols)  # number of temperatures (targets and input)
        n_conds = int(0.5 * n_temps * (n_temps - 1))  # number of thermal conductances
        # conductance net sub-NN
        self.conductance_net = nn.Sequential(
            nn.Linear(len(input_cols) + self.output_size, n_conds), 
            nn.ReLU(),         # Activation function added by Yuping
            nn.Linear(n_conds, n_conds), # Additional Layer added by Yuping
            nn.Sigmoid()
        )
        # populate adjacency matrix. It is used for indexing the conductance sub-NN output
        self.adj_mat = np.zeros((n_temps, n_temps), dtype=int)
        adj_idx_arr = np.ones_like(self.adj_mat)
        triu_idx = np.triu_indices(n_temps, 1)
        adj_idx_arr = adj_idx_arr[triu_idx].ravel()
        self.adj_mat[triu_idx] = np.cumsum(adj_idx_arr) - 1
        self.adj_mat += self.adj_mat.T
        self.adj_mat = torch.from_numpy(self.adj_mat[: self.output_size, :]).type(
            torch.int64
        )  # crop
        self.n_temps = n_temps

        # power loss sub-NN
        self.ploss = nn.Sequential(
            nn.Linear(len(input_cols) + self.output_size, 16),
            nn.ReLU(),         # Activation function added by Yuping
            nn.Linear(16, 16), # Additional Layer added by Yuping
            nn.Tanh(),
            nn.Linear(16, self.output_size),
        )

        self.temp_idcs = [i for i, x in enumerate(input_cols) if x in temperature_cols]
        self.nontemp_idcs = [
            i
            for i, x in enumerate(input_cols)
            if x not in temperature_cols + ["profile_id"]
        ]

    def forward(self, inp: Tensor, hidden: Tensor) -> Tuple[Tensor, Tensor]:
        prev_out = hidden
        temps = torch.cat([prev_out, inp[:, self.temp_idcs]], dim=1)
        sub_nn_inp = torch.cat([inp, prev_out], dim=1)
        conducts = torch.abs(self.conductance_net(sub_nn_inp))
        power_loss = torch.abs(self.ploss(sub_nn_inp))
        temp_diffs = torch.sum(
            (temps.unsqueeze(1) - prev_out.unsqueeze(-1)) * conducts[:, self.adj_mat],
            dim=-1,
        )
        out = prev_out + self.sample_time * torch.exp(self.caps) * (
            temp_diffs + power_loss
        )
        return prev_out, torch.clip(out, -1, 5)

## Training

In [0]:
model = torch.jit.script(DiffEqLayer(TNNCell).to(device))
loss_func = nn.MSELoss(reduction="none")
opt = optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 500 #100
tbptt_size = 100 #512

n_batches = np.ceil(train_tensor.shape[0] / tbptt_size).astype(int)
with tqdm(desc="Training", total=n_epochs) as pbar:
    for epoch in range(n_epochs):
        # first state is ground truth temperature data
        hidden = train_tensor[0, :, -len(target_cols) :]

        # propagate batch-wise through data set
        for i in range(n_batches):
            model.zero_grad()
            output, hidden = model(
                train_tensor[
                    i * tbptt_size : (i + 1) * tbptt_size, :, : len(input_cols)
                ],
                hidden.detach(),
            )
            loss = loss_func(
                output,
                train_tensor[
                    i * tbptt_size : (i + 1) * tbptt_size, :, -len(target_cols) :
                ],
            )
            # sample_weighting
            loss = (
                (
                    loss
                    * train_sample_weights[
                        i * tbptt_size : (i + 1) * tbptt_size, :, None
                    ]
                    / train_sample_weights[
                        i * tbptt_size : (i + 1) * tbptt_size, :
                    ].sum()
                )
                .sum()
                .mean()
            )
            loss.backward()
            opt.step()

        # reduce learning rate
        if epoch == int(n_epochs*0.75):
            for group in opt.param_groups:
                group["lr"] *= 0.5
        pbar.update()
        pbar.set_postfix_str(f"loss: {loss.item():.2e}")

Training:  14%|█▎        | 68/500 [03:45<24:52,  3.45s/it, loss: 3.55e-03]

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:717)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## Testing

In [0]:
# model saving and loading
mdl_path = Path.cwd() / 'data' / 'models'
mdl_path.mkdir(exist_ok=True, parents=True)
# mdl_file_path = mdl_path / 'tnn_jit_torch.pt'
mdl_file_path = mdl_path / 'tnn_jit_torch_STM_RTM_Oil_Joint_EFAD.pt'
# mdl_file_path = mdl_path / 'tnn_jit_torch_RTM_EFAD.pt'
model.save(mdl_file_path)  # save
model = torch.jit.load(mdl_file_path)  # load
model.eval()

In [0]:
# evaluate against test set
with torch.no_grad():
    pred, hidden = model(
        test_tensor[:, :, : len(input_cols)], test_tensor[0, :, -len(target_cols) :]
    )
    pred = pred.cpu().numpy() * 200  # denormalize

## Visualize Performance
We see the performance for one trial, i.e., one run of random initialized weights.

Usually, running multiple trials with different random number generator seeds yields even better results.


In [0]:
fig, axes = plt.subplots(len(test_profiles), len(target_cols), figsize=(20, 10))


# 保证 axes 是二维数组
if len(test_profiles) == 1 and len(target_cols) == 1:
    axes = np.array([[axes]])
elif len(test_profiles) == 1:
    axes = axes[np.newaxis, :]
elif len(target_cols) == 1:
    axes = axes[:, np.newaxis]


for i, (pid, y_test) in enumerate(
    data.loc[data.profile_id.isin(test_profiles), target_cols + ["profile_id"]].groupby(
        "profile_id"
    )
):
    y_test *= 200
    profile_pred = pred[: len(y_test), i, :]
    for j, col in enumerate(target_cols):
        ax = axes[i, j]
        ax.plot(
            y_test.loc[:, col].reset_index(drop=True),
            color="tab:green",
            label="Ground truth",
        )
        ax.plot(profile_pred[:, j], color="tab:blue", label="Prediction")
        ax.text(
            x=0.5,
            y=0.8,
            s=f"MSE: {((profile_pred[:, j] - y_test.loc[:, col])**2).sum() / len(profile_pred):.3f} K²\nmax.abs.:{(profile_pred[:, j]-y_test.loc[:, col]).abs().max():.1f} K",
            transform=ax.transAxes,
        )
        if j == 0:
            ax.set_ylabel(f"Profile {pid}\n Temp. in °C")
            if i == 0:
                ax.legend()
        if i == len(test_profiles) - 1:
            ax.set_xlabel(f"Iters")
        elif i == 0:
            ax.set_title(col)

The performance that is achievable by the hybridization of LPTNs with neural networks is unprecedented and not achievable by pure LPTN or pure black-box ML models.
Note that the visualized performance stems from training a TNN from scratch once. All neural networks are initialized randomly when their training by gradient descent begins.
This means that better performance can be easily achieved by repeating this experiment since the convergence into better local minima becomes likely.

In [0]:
# evaluate against test set
with torch.no_grad():
    pred_train, hidden_train = model(
        train_tensor[:, :, : len(input_cols)], train_tensor[0, :, -len(target_cols) :]
    )
    pred_train = pred_train.cpu().numpy() * 200  # denormalize

data_train = data.copy()
data_train.loc[data_train.profile_id.isin(train_profiles), target_cols] *=200 # denormalize
results_cmbd_merged = pd.DataFrame()
target_estd_cols = [col + '_Estd' for col in target_cols]
for i, (pid, y_train) in enumerate(
    data_train.loc[data_train.profile_id.isin(train_profiles), target_cols + input_cols + ["profile_id"]].groupby(
        "profile_id"
    )
):
    results_ref = y_train.reset_index(drop=True).copy()
    profile_pred = pred_train[: len(y_train), i, :]
    results_estd = pd.DataFrame(profile_pred, columns=target_estd_cols) 
    results_cmbd = pd.concat([results_ref, results_estd], axis=1)
    results_cmbd_merged = pd.concat([results_cmbd_merged, results_cmbd], ignore_index=True)
    

# Create subplots with 5 rows and 1 column, with third row having dual Y-axis
fig = make_subplots(
    rows=5, 
    cols=1, 
    shared_xaxes=True, 
    vertical_spacing=0.08,
    specs=[
        [{}],         # Row 1: Single Y-axis
        [{}],         # Row 2: Single Y-axis
        [{"secondary_y": True}],  # Row 3: Dual Y-axis
        [{"secondary_y": True}],  # Row 4: Dual Y-axis
        [{}]          # Row 5: Single Y-axis
    ]
)

# Row 1: Actual and estimated values
for col in target_cols:
    fig.add_trace(go.Scatter(
        x=results_cmbd_merged.index,
        y=results_cmbd_merged[col],
        mode='lines',
        name=col
    ), row=1, col=1)
    
for col in target_estd_cols:
    fig.add_trace(go.Scatter(
        x=results_cmbd_merged.index,
        y=results_cmbd_merged[col],
        mode='lines',
        name=col
    ), row=1, col=1)

# Row 2: Difference between estimated and actual values
for col_actual, col_estd in zip(target_cols, target_estd_cols):
    fig.add_trace(go.Scatter(
        x=results_cmbd_merged.index,
        y=results_cmbd_merged[col_estd] - results_cmbd_merged[col_actual],
        mode='lines',
        name=f"{col_estd} - {col_actual}"
    ), row=2, col=1)

# Row 3: tqEm and nEm with dual Y-axis
# tqEm - Left axis
fig.add_trace(go.Scatter(
    x=results_cmbd_merged.index,
    y=results_cmbd_merged['tqEmFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm'] * MeasurementData_Merged['tqEmFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm'].abs().max(),
    mode='lines',
    name='tqEm'
), row=3, col=1, secondary_y=False)

# nEm - Right axis
fig.add_trace(go.Scatter(
    x=results_cmbd_merged.index,
    y=results_cmbd_merged['nEmFild100ms.Rec_10ms_Fild.pp_rbe_CddAgEm_10ms_Fild.rbe_CddRslvr'] * MeasurementData_Merged['nEmFild100ms.Rec_10ms_Fild.pp_rbe_CddAgEm_10ms_Fild.rbe_CddRslvr'].abs().max(),
    mode='lines',
    name='nEm'
), row=3, col=1, secondary_y=True)

# Row 4: tCoolt and VfCoolt with dual Y-axis
# tqEm - Left axis
fig.add_trace(go.Scatter(
    x=results_cmbd_merged.index,
    y=results_cmbd_merged['R_CW_tCooltIvtrOut'] * MeasurementData_Merged['R_CW_tCooltIvtrOut'].abs().max(),
    mode='lines',
    name='tCooltInvtrOut'
), row=4, col=1, secondary_y=False)

# # VfCoolt - Right axis
# fig.add_trace(go.Scatter(
#     x=results_cmbd_merged.index,
#     y=,
#     mode='lines',
#     name='VfCoolt'
# ), row=4, col=1, secondary_y=True)

# Row 5: Profile ID
fig.add_trace(go.Scatter(
    x=results_cmbd_merged.index,
    y=results_cmbd_merged['profile_id'],
    mode='lines',
    name="Profile ID"
), row=5, col=1)

# Layout configuration
fig.update_layout(
    height=1200,
    # width=1200,
    title_text="Actual vs Estimated Values Comparison",
    showlegend=True
)

# Configure axis titles
fig.update_yaxes(title_text="Act/Estd Temp (°C)", row=1, col=1)
fig.update_yaxes(title_text="Est. Err. (°C)", row=2, col=1)
fig.update_yaxes(title_text="tqEm (Nm)", row=3, col=1)
fig.update_yaxes(
    title_text="nEm (rpm)", 
    overlaying="y3",         # Overlay on row 3's primary Y-axis
    side="right",            # Position on right side
    row=3, col=1,
    secondary_y=True
)
fig.update_yaxes(title_text="tCooltIvtrOut (°C)", row=4, col=1)
fig.update_yaxes(title_text="Profile ID", row=5, col=1)

fig.show()


# 假设你已经创建了一个 Plotly 图表对象 fig
pio.write_html(fig, file=f'{target_cols}.html', auto_open=False)

# Plot Histogram figures
for col_actual, col_estd in zip(target_cols, target_estd_cols):
    # Calculate error distribution
    error = results_cmbd_merged[col_estd] - results_cmbd_merged[col_actual]
    mu, std = norm.fit(error)
    xmin, xmax = error.min(), error.max()
    
    # Create histogram
    hist = go.Histogram(
        x=error,
        histnorm='probability density',  # Normalize to PDF
        nbinsx=100,                      # Match original bin count
        marker=dict(color='green', opacity=0.6, line=dict(width=1, color='black')),
        name='Error Distribution',
        hoverinfo='x+y',
        showlegend=True
    )
    
    # Create fitted normal curve
    x_fit = np.linspace(xmin, xmax, 100)
    y_fit = norm.pdf(x_fit, mu, std)
    fit_curve = go.Scatter(
        x=x_fit,
        y=y_fit,
        mode='lines',
        line=dict(color='black', width=2),
        name='Normal Fit',
        hoverinfo='none'
    )
    
    # Create statistical reference lines
    lines = [
        # Mean line (red dashed)
        go.Scatter(
            x=[mu, mu],
            y=[0, norm.pdf(mu, mu, std)*1.1],  # Height = 110% of peak density
            mode='lines',
            line=dict(color='red', width=2, dash='dash'),
            name=f'μ = {mu:.2f}',
            hoverinfo='x+name'
        ),
        # +3σ line (orange dotted)
        go.Scatter(
            x=[mu+3*std, mu+3*std],
            y=[0, norm.pdf(mu, mu, std)*1.1],
            mode='lines',
            line=dict(color='orange', width=2, dash='dot'),
            name=f'μ+3σ = {mu+3*std:.2f}',
            hoverinfo='x+name'
        ),
        # -3σ line (orange dotted)
        go.Scatter(
            x=[mu-3*std, mu-3*std],
            y=[0, norm.pdf(mu, mu, std)*1.1],
            mode='lines',
            line=dict(color='orange', width=2, dash='dot'),
            name=f'μ-3σ = {mu-3*std:.2f}',
            hoverinfo='x+name'
        ),
        # Min error line (blue dotted)
        go.Scatter(
            x=[xmin, xmin],
            y=[0, norm.pdf(mu, mu, std)*1.1],
            mode='lines',
            line=dict(color='blue', width=2, dash='dot'),
            name=f'Min Error = {xmin:.2f}',
            hoverinfo='x+name'
        ),
        # Max error line (blue dotted)
        go.Scatter(
            x=[xmax, xmax],
            y=[0, norm.pdf(mu, mu, std)*1.1],
            mode='lines',
            line=dict(color='blue', width=2, dash='dot'),
            name=f'Max Error = {xmax:.2f}',
            hoverinfo='x+name'
        )
    ]
    
    # Combine all traces
    data2plot = [hist, fit_curve] + lines
    
    # Create layout with interactive features
    layout = go.Layout(
        title=f"Fitting results {col_actual}: μ = {mu:.2f}, σ = {std:.2f}",
        xaxis=dict(title='Estd-Measd (K)', showgrid=True),
        yaxis=dict(title='Probability Density', showgrid=True),
        hovermode='closest',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=500,
        width=800,
        margin=dict(t=150, b=80),
        template='plotly_white'
    )
    
    # Create and show figure
    fig = go.Figure(data=data2plot, layout=layout)
    fig.show()
    # 假设你已经创建了一个 Plotly 图表对象 fig
    pio.write_html(fig, file=f'{col_actual}_Histogram.html', auto_open=False)

## Back-up Scripts

### Hyper Parameters Optimization (optuna method based)

In [0]:
# 自定义正弦激活层
class SinusLayer(nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.sin(x)

# 激活函数映射
def get_activation(activation_name: str) -> nn.Module:
    activation_dict = {
        "Sigmoid": nn.Sigmoid(),
        "tanh": nn.Tanh(),
        "linear": nn.Identity(),
        "ReLU": nn.ReLU(),
        "biased Elu": nn.ELU(alpha=1.0),
        "sinus": SinusLayer()
    }
    return activation_dict[activation_name]

# TNNCell定义（支持动态结构和激活函数）
class TNNCell(nn.Module):
    def __init__(self, cond_net_layers, cond_net_units, cond_activation, 
                 ploss_net_layers, ploss_net_units, ploss_activation):
        super().__init__()
        self.sample_time = 0.5
        self.output_size = len(target_cols)
        self.caps = nn.Parameter(torch.Tensor(self.output_size))
        nn.init.normal_(self.caps, mean=-9.2, std=0.5)
        
        n_temps = len(temperature_cols)
        n_conds = int(0.5 * n_temps * (n_temps - 1))
        
        # 动态构建conductance_net
        cond_layers = []
        input_dim = len(input_cols) + self.output_size
        for units in cond_net_units:
            cond_layers.append(nn.Linear(input_dim, units))
            cond_layers.append(get_activation(cond_activation))
            input_dim = units
        cond_layers.append(nn.Linear(input_dim, n_conds))
        cond_layers.append(nn.Sigmoid())
        self.conductance_net = nn.Sequential(*cond_layers)
        
        # 动态构建ploss_net
        ploss_layers = []
        input_dim = len(input_cols) + self.output_size
        for i, units in enumerate(ploss_net_units):
            ploss_layers.append(nn.Linear(input_dim, units))
            if i < len(ploss_net_units) - 1:
                ploss_layers.append(get_activation(ploss_activation))
            input_dim = units
        ploss_layers.append(nn.Linear(input_dim, self.output_size))
        self.ploss = nn.Sequential(*ploss_layers)
        
        # 其余初始化代码
        self.adj_mat = np.zeros((n_temps, n_temps), dtype=int)
        triu_idx = np.triu_indices(n_temps, 1)
        adj_idx_arr = np.ones_like(self.adj_mat)
        adj_idx_arr = adj_idx_arr[triu_idx].ravel()
        self.adj_mat[triu_idx] = np.cumsum(adj_idx_arr) - 1
        self.adj_mat += self.adj_mat.T
        self.adj_mat = torch.from_numpy(self.adj_mat[:self.output_size, :]).type(torch.int64)
        self.temp_idcs = [i for i, x in enumerate(input_cols) if x in temperature_cols]
        self.nontemp_idcs = [i for i, x in enumerate(input_cols) if x not in temperature_cols + ["profile_id"]]

    def forward(self, inp: Tensor, hidden: Tensor) -> Tuple[Tensor, Tensor]:
        prev_out = hidden
        temps = torch.cat([prev_out, inp[:, self.temp_idcs]], dim=1)
        sub_nn_inp = torch.cat([inp, prev_out], dim=1)
        conducts = torch.abs(self.conductance_net(sub_nn_inp))
        power_loss = torch.abs(self.ploss(sub_nn_inp))
        temp_diffs = torch.sum(
            (temps.unsqueeze(1) - prev_out.unsqueeze(-1)) * conducts[:, self.adj_mat],
            dim=-1,
        )
        out = prev_out + self.sample_time * torch.exp(self.caps) * (temp_diffs + power_loss)
        return prev_out, torch.clip(out, -1, 5)

# DiffEqLayer定义（支持TorchScript）
class DiffEqLayer(ScriptModule):
    def __init__(self, cell_module):
        super().__init__()
        self.cell = cell_module
        
    @script_method
    def forward(self, input: Tensor, state: Tensor) -> Tuple[Tensor, Tensor]:
        inputs = input.unbind(0)
        outputs = []
        for i in range(len(inputs)):
            out, state = self.cell(inputs[i], state)
            outputs.append(out)
        return torch.stack(outputs), state

# Optuna目标函数
def objective(trial: optuna.Trial) -> float:
    # 1. 超参数采样
    cond_net_layers = trial.suggest_int('cond_net_layers', 1, 3)
    cond_net_units = [trial.suggest_int(f'cond_net_units_{i}', 2, 30) 
                     for i in range(cond_net_layers)]
    
    ploss_net_layers = trial.suggest_int('ploss_net_layers', 1, 3)
    ploss_net_units = [trial.suggest_int(f'ploss_net_units_{i}', 2, 30) 
                      for i in range(ploss_net_layers)]
    
    cond_activation = trial.suggest_categorical(
        "cond_activation", 
        ["Sigmoid", "tanh", "linear", "ReLU", "biased Elu", "sinus"]
    )
    ploss_activation = trial.suggest_categorical(
        "ploss_activation", 
        ["Sigmoid", "tanh", "linear", "ReLU", "biased Elu", "sinus"]
    )
    
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'NAdam'])
    tbptt_size = trial.suggest_int('tbptt_size', 50, 512)

    # 2. 构建模型（支持TorchScript）
    tnn_cell = TNNCell(
        cond_net_layers, cond_net_units, cond_activation,
        ploss_net_layers, ploss_net_units, ploss_activation
    ).to(device)
    
    model = DiffEqLayer(tnn_cell).to(device)
    
    # 3. 选择优化器
    if optimizer_name == 'Adam':
        opt = optim.Adam(model.parameters(), lr=lr)
    else:
        opt = optim.NAdam(model.parameters(), lr=lr)
    
    # 4. 训练与评估（使用训练集本身）
    n_epochs = 100
    loss_func = nn.MSELoss(reduction="none")
    best_train_loss = float('inf')
    
    for epoch in range(n_epochs):
        hidden = train_tensor[0, :, -len(target_cols):].detach()
        n_batches = int(np.ceil(train_tensor.shape[0] / tbptt_size))
        epoch_loss = 0.0
        
        model.train()
        for i in range(n_batches):
            opt.zero_grad()
            
            # 前向传播
            output, hidden = model(
                train_tensor[i*tbptt_size : (i+1)*tbptt_size, :, :len(input_cols)],
                hidden.detach()
            )
            
            # 计算训练损失
            loss = loss_func(
                output,
                train_tensor[i*tbptt_size : (i+1)*tbptt_size, :, -len(target_cols):]
            )
            
            # 加权损失计算
            loss = (loss * train_sample_weights[i*tbptt_size:(i+1)*tbptt_size, :, None]).sum() 
            loss /= train_sample_weights[i*tbptt_size:(i+1)*tbptt_size, :].sum() + 1e-8  # 防止除零
            
            # 反向传播
            loss.backward()
            opt.step()
            epoch_loss += loss.item()
        
        # 计算平均训练损失
        avg_epoch_loss = epoch_loss / n_batches
        
        # 报告训练损失给Optuna
        trial.report(avg_epoch_loss, epoch)
        
        # 剪枝逻辑（基于训练损失）
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        # 记录最佳训练损失
        if avg_epoch_loss < best_train_loss:
            best_train_loss = avg_epoch_loss
    
    return best_train_loss

# 创建Optuna研究
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=10
    )
)

# 启动优化
study.optimize(objective, n_trials=200, show_progress_bar=True)

# 输出结果
print("最佳训练损失:", study.best_value)
print("最佳参数组合:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# 使用最佳参数训练最终模型
best_params = study.best_params
final_tnn_cell = TNNCell(
    cond_net_layers=best_params['cond_net_layers'],
    cond_net_units=[best_params[f'cond_net_units_{i}'] for i in range(best_params['cond_net_layers'])],
    cond_activation=best_params['cond_activation'],
    ploss_net_layers=best_params['ploss_net_layers'],
    ploss_net_units=[best_params[f'ploss_net_units_{i}'] for i in range(best_params['ploss_net_layers'])],
    ploss_activation=best_params['ploss_activation']
).to(device)

final_model = torch.jit.script(DiffEqLayer(final_tnn_cell)).to(device)
# 从Optuna最佳参数中提取优化器名称
optimizer_name = best_params['optimizer']
# 动态创建优化器
if optimizer_name == 'Adam':
    opt = optim.Adam(final_model.parameters(), lr=best_params['lr'])
else:  # 对应NAdam
    opt = optim.NAdam(final_model.parameters(), lr=best_params['lr'])
# 最终训练
n_epochs = 500
tbptt_size = best_params['tbptt_size']

with tqdm(desc="Final Training", total=n_epochs) as pbar:
    for epoch in range(n_epochs):
        hidden = train_tensor[0, :, -len(target_cols):].detach()
        n_batches = int(np.ceil(train_tensor.shape[0] / tbptt_size))
        
        for i in range(n_batches):
            opt.zero_grad()
            output, hidden = final_model(
                train_tensor[i*tbptt_size : (i+1)*tbptt_size, :, :len(input_cols)],
                hidden.detach()
            )
            
            loss = loss_func(
                output,
                train_tensor[i*tbptt_size : (i+1)*tbptt_size, :, -len(target_cols):]
            )
            
            loss = (loss * train_sample_weights[i*tbptt_size:(i+1)*tbptt_size, :, None]).sum()
            loss /= train_sample_weights[i*tbptt_size:(i+1)*tbptt_size, :].sum()
            
            loss.backward()
            opt.step()
        
        # 学习率衰减
        if epoch == int(n_epochs*0.75):
            for group in opt.param_groups:
                group["lr"] *= 0.5
        
        pbar.update()
        pbar.set_postfix_str(f"loss: {loss.item():.2e}")

# model saving and loading
mdl_path = Path.cwd() / 'data' / 'models'
mdl_path.mkdir(exist_ok=True, parents=True)
# mdl_file_path = mdl_path / 'tnn_jit_torch.pt'
mdl_file_path = mdl_path / 'tnn_jit_torch_STM_RTM_Oil_Joint_EFAD.pt'
# mdl_file_path = mdl_path / 'tnn_jit_torch_RTM_EFAD.pt'
final_model.save(mdl_file_path)  # save
final_model = torch.jit.load(mdl_file_path)  # load
final_model.eval()

[I 2025-06-16 17:18:18,379] A new study created in memory with name: no-name-dcbacc97-0406-42de-ba9f-baea6b10b6e1


  0%|          | 0/200 [00:00<?, ?it/s]

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:717)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can